<a href="https://colab.research.google.com/github/jeon-maker/Team4_food-project/blob/main/food_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#food detection
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report

In [2]:
image_dir = Path('/content/drive/MyDrive/food_detection/images')


In [3]:
filepaths = list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))


In [4]:
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

In [5]:
images = pd.concat([filepaths, labels], axis=1)

In [6]:
category_samples = []
for category in images['Label'].unique():
    category_slice = images.query("Label == @category")
    category_samples.append(category_slice.sample(100, random_state=1,replace=True))
image_df = pd.concat(category_samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [7]:
image_df

,Filepath,Label
0,/content/drive/MyDrive/food_detection/images/p...,paella
1,/content/drive/MyDrive/food_detection/images/l...,lobster_bisque
2,/content/drive/MyDrive/food_detection/images/c...,cheesecake
3,/content/drive/MyDrive/food_detection/images/d...,dumplings
4,/content/drive/MyDrive/food_detection/images/f...,french_fries
...,...,...
7195,/content/drive/MyDrive/food_detection/images/b...,breakfast_burrito
7196,/content/drive/MyDrive/food_detection/images/g...,guacamole
7197,/content/drive/MyDrive/food_detection/images/f...,foie_gras
7198,/content/drive/MyDrive/food_detection/images/b...,baklava


In [8]:
image_df['Label'].value_counts()

paella            100
lobster_bisque    100
frozen_yogurt     100
chicken_wings     100
deviled_eggs      100
                 ... 
filet_mignon      100
lasagna           100
hot_dog           100
chocolate_cake    100
edamame           100
Name: Label, Length: 72, dtype: int64

Train Test Split


In [9]:
train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)


In [10]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [11]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)



Found 4032 validated image filenames belonging to 72 classes.
Found 1008 validated image filenames belonging to 72 classes.
Found 2160 validated image filenames belonging to 72 classes.


In [16]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

In [13]:
#""" from keras.models import Sequential"""
#""" from keras import layers"""
#""" from keras.optimizers import RMSprop"""

from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

In [24]:

inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(72, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

Training

In [25]:
with tf.device('/device:GPU:0'):
    model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )
    
    history = model.fit(
      train_images,
      validation_data=val_images,
      epochs=10,
      callbacks=[
          tf.keras.callbacks.EarlyStopping(
              monitor='val_loss',
              patience=3,
              restore_best_weights=True
          )
      ]
    )


Epoch 1/10
126/126 [==============================] - 2987s 24s/step - loss: 3.4816 - accuracy: 0.1815 - val_loss: 2.5682 - val_accuracy: 0.3810
Epoch 2/10
126/126 [==============================] - 38s 304ms/step - loss: 2.0223 - accuracy: 0.4732 - val_loss: 2.1852 - val_accuracy: 0.4365
Epoch 3/10
126/126 [==============================] - 38s 301ms/step - loss: 1.4613 - accuracy: 0.6146 - val_loss: 2.0733 - val_accuracy: 0.4871
Epoch 4/10
126/126 [==============================] - 38s 302ms/step - loss: 1.1192 - accuracy: 0.6877 - val_loss: 2.0148 - val_accuracy: 0.5159
Epoch 5/10
126/126 [==============================] - 39s 307ms/step - loss: 0.8194 - accuracy: 0.7783 - val_loss: 1.9947 - val_accuracy: 0.5258
Epoch 6/10
126/126 [==============================] - 38s 302ms/step - loss: 0.5939 - accuracy: 0.8472 - val_loss: 2.1669 - val_accuracy: 0.5129
Epoch 7/10
126/126 [==============================] - 37s 297ms/step - loss: 0.4182 - accuracy: 0.9003 - val_loss: 2.1553 - val_ac